### **1. 지하수 데이터 수집**

In [ ]:
#필요한 라이브러리 포함
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
from datetime import datetime, date, time
warnings.filterwarnings('ignore')

In [ ]:
#엑셀 데이터 파일 읽어오기
data1 = pd.read_excel('./Data/대전태평(암반)/대전태평(암반)_2016_hr.xlsx')
data1

In [ ]:
#엑셀 데이터 파일 읽어오기
data2 = pd.read_excel('./Data/대전태평(암반)/대전태평(암반)_2015_hr.xlsx')
data2

In [ ]:
#데이터 결합
data = pd.concat([data1, data2])
data = data.reset_index(drop=True)
data

In [ ]:
#데이터 파일 경로와 파일 이름을 리스트로 저장
path = './Data/대전태평(암반)/'
filelist = os.listdir(path)
filelistpy = [file for file in filelist if file.endswith('.xlsx')]
filelistpy

In [ ]:
#파일이름리스트로 파일 여러개를 반복하여 읽어오기
df = pd.DataFrame()    #빈 데이터프레임 생성
for i in filelistpy:  #파일이름리스트 요소 만큼 읽고 결합 반복
  d = pd.read_excel(path + i)
  df = pd.concat([df, d])
df

In [ ]:
df = df.reset_index(drop=True)
df

In [ ]:
df.info()

In [ ]:
df['Date'] = pd.to_datetime(df['날짜'],format='%Y%m%d') +\
             pd.to_timedelta(df['시간'].astype(int),unit='h')

df.set_index(df['Date'], inplace=True)
df.head(3)

In [ ]:
df1 = df[['수온(℃)', '수위(el.m)','EC(㎲/㎝)']]
df1.columns = ['temp', 'level', 'EC']
df1.head(3)

In [ ]:
df1.isnull().sum()

In [ ]:
df1.to_csv('./대전지하수.csv', encoding='cp949')

### **2. 이상치 확인 및 제거**

In [ ]:
df = pd.read_csv('./대전지하수.csv', index_col='Date', parse_dates=True, encoding='cp949')
df.describe()

In [ ]:
df.hist(bins=50, figsize=(10,6))
plt.show()

In [ ]:
df.boxplot()

In [ ]:
plt.subplot(1, 3, 1)
df.boxplot(column='temp', return_type='both')
plt.subplot(1, 3, 2)
df.boxplot(column='level', return_type='both')
plt.subplot(1, 3, 3)
df.boxplot(column='EC', return_type='both')
plt.show()

In [ ]:
#코드 8-24
plt.subplot(3, 1, 1)
df['temp'].plot()
plt.subplot(3, 1, 2)
df['level'].plot()
plt.subplot(3, 1, 3)
df['EC'].plot()
plt.show()

In [ ]:
q3_level = df['level'].quantile(q=0.75)
q1_level = df['level'].quantile(q=0.25)
iqr_level = q3_level - q1_level
print(iqr_level)

In [ ]:
upper_level = q3_level + 1.5 * iqr_level
lower_level = q1_level - 1.5 * iqr_level
print(upper_level, '/', lower_level)
print((df['level'] > upper_level).sum())
print((df['level'] < lower_level).sum())

In [ ]:
df_iqr_level = df[(df['level'] < upper_level) & (df['level'] > lower_level)]
df_iqr_level['level'].plot()

In [ ]:
#사분위수와 IQR 구하기
q3_temp = df['temp'].quantile(q=0.75)
q1_temp = df['temp'].quantile(q=0.25)
iqr_temp = q3_temp-q1_temp
print('IQR:', iqr_temp)
#지하수온 상한값, 하한값, 이상치 개수 구하기
upper_temp = q3_temp + iqr_temp * 1.5
lower_temp = q1_temp - iqr_temp * 1.5
print(upper_temp, '/', lower_temp)
print((df['temp'] > upper_temp).sum())
print((df['temp'] < lower_temp).sum())

In [ ]:
#사분위수와 IQR 구하기
q3_ec = df['EC'].quantile(q=0.75)
q1_ec = df['EC'].quantile(q=0.25)
iqr_ec = q3_ec - q1_ec
print('IQR:', iqr_ec)
#지하수온 상한값, 하한값, 이상치 개수 구하기
upper_ec = q3_ec + iqr_ec * 1.5
lower_ec = q1_ec - iqr_ec * 1.5
print(upper_ec, '/', lower_ec)
print((df['EC'] > upper_ec).sum())
print((df['EC'] < lower_ec).sum())

In [ ]:
df_iqr = df[(df['temp'] < upper_temp) & (df['temp'] > lower_temp) &\
            (df['level'] < upper_level) & (df['level'] > lower_level) &\
             (df['EC'] < upper_ec) & (df['EC'] > lower_ec)]

In [ ]:
df_iqr.describe()

In [ ]:
df_iqr.hist(bins=50, figsize=(10,6))
plt.show()

In [ ]:
plt.subplot(1, 3, 1)
df_iqr.boxplot(column='temp', return_type='both')
plt.subplot(1, 3, 2)
df_iqr.boxplot(column='level', return_type='both')
plt.subplot(1, 3, 3)
df_iqr.boxplot(column='EC', return_type='both')
plt.show()

In [ ]:
plt.subplot(3, 1, 1)
df_iqr['temp'].plot()
plt.subplot(3, 1, 2)
df_iqr['level'].plot()
plt.subplot(3, 1, 3)
df_iqr['EC'].plot()
plt.show()

In [ ]:
df_iqr.to_csv('대전지하수_정제.csv', encoding='cp949')

### **3. 표준화와 정규화**

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('./대전지하수_정제.csv')
df.head()

In [ ]:
df = df[['level', 'EC']]
df.describe()

In [ ]:
df.hist()
plt.show()

In [ ]:
df1 = df.copy()
df1.head(3)

In [ ]:
df1['level_z_score'] = (df1['level'] - df1['level'].mean()) / df1['level'].std()
df1['EC_z_score'] = (df1['EC'] - df1['EC'].mean()) / df1['EC'].std()
df1.head(3)

In [ ]:

df1.describe()

In [ ]:
df1.hist(figsize=(8,5))
plt.show()

In [ ]:
df2 = df.copy()
df2_standard = (df2-df2.mean()) / df2.std()
df2_standard.head(3)

In [ ]:
df3 = df.copy()

df3['level_minmax'] = (df3['level']-df3['level'].min()) / (df3['level'].max()\
                                                           -df3['level'].min())
df3['EC_minmax'] = (df3['EC'] - df3['EC'].min()) / (df3['EC'].max() - df3['EC'].min())
df3.head(3)

In [ ]:
df3.describe()

In [ ]:
df3.hist(figsize=(8,6))
plt.show()